<a href="https://colab.research.google.com/github/Krahjotdaan/MachineLearning/blob/main/MLCompetition2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB, ComplementNB
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier

In [ ]:
!pip3 install catboost

In [ ]:
import xgboost as xgb
import catboost as cat
import lightgbm as light

Откроем файл с данными.

In [ ]:
df = pd.read_csv('credit_train.csv')

y_train = df['opened']
X_train = df.drop(columns=['gender', 'marital_status', 'education', 'job_pos', 'region', 'opened'])
X_train['credit_sum'] = X_train['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_train = X_train.fillna(-1)

In [ ]:
df_test = pd.read_csv('credit_test.csv')

X_test = df_test.drop(columns=['gender', 'marital_status', 'education', 'job_pos', 'region'])
X_test['credit_sum'] = X_test['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_test = X_test.fillna(-1)

In [ ]:
df

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income,opened
0,26031,M,24,UNM,"24000,00",10,1.60,SCH,2.0,0.0,SPC,"0,728851",ОБЛ ЛЕНИНГРАДСКАЯ,35000.0,True
1,148877,M,23,UNM,"16339,00",18,1.10,SCH,2.0,0.0,SPC,"0,347385",МОСКВА Г,38000.0,True
2,142019,M,54,UNM,"15652,00",10,1.32,GRD,2.0,0.0,SPC,"0,349705",ОБЛ МОСКОВСКАЯ,38000.0,True
3,146816,F,52,UNM,"16560,00",10,1.10,SCH,1.0,0.0,SPC,"0,359232",РЕСП МОРДОВИЯ,28000.0,False
4,27699,F,63,WID,"9078,00",18,1.10,SCH,4.0,0.0,SPC,"0,529368",ЧЕЛЯБИНСКАЯ ОБЛ,30000.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114394,119880,M,29,MAR,"29678,00",10,1.60,GRD,1.0,0.0,SPC,"0,721156",ОМСКАЯ ОБЛАСТЬ,25000.0,False
114395,103695,M,29,MAR,"23068,00",15,1.10,GRD,3.0,0.0,SPC,"0,566393",ТЮМЕНСКАЯ ОБЛ,30000.0,True
114396,131933,F,45,MAR,"16418,00",10,1.10,SCH,1.0,0.0,UMN,"0,327380",ОБЛ БРЯНСКАЯ,30400.0,False
114397,146868,F,49,MAR,"12169,00",10,1.40,SCH,1.0,0.0,SPC,"0,467395",ТАМБОВСКАЯ ОБЛ,16000.0,False


In [ ]:
df_test

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income
0,39162,F,18,UNM,"39469,00",6,1.44,GRD,NaN,NaN,SPC,"0,424628",САНКТ-ПЕТЕРБУРГ Г,30000.0
1,10673,F,63,WID,"16693,00",10,1.32,GRD,2.0,0.0,SPC,"0,311945",ОБЛ ЛЕНИНГРАДСКАЯ,21250.0
2,121409,M,42,MAR,"60260,00",10,1.10,GRD,5.0,0.0,SPC,"0,421982",МОСКВА Г,150000.0
3,23504,M,41,MAR,"32479,00",12,1.10,GRD,1.0,0.0,DIR,"0,406572",ТАТАРСТАН РЕСП,150000.0
4,69269,F,24,UNM,"8159,00",12,1.10,SCH,2.0,0.0,SPC,"0,418243",ТВЕРСКАЯ ОБЛ,16000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56342,14148,F,21,UNM,"40980,00",24,1.90,GRD,1.0,0.0,SPC,"0,429402",САМАРСКАЯ,22000.0
56343,153732,M,56,MAR,"16009,00",10,1.10,SCH,0.0,0.0,SPC,"0,494229",САМАРСКАЯ ОБЛ,29000.0
56344,37647,M,54,MAR,"22448,00",12,1.10,SCH,1.0,0.0,UMN,"0,312089",РОСТОВСКАЯ ОБЛ,26500.0
56345,92808,F,27,UNM,"13829,00",10,1.90,GRD,2.0,0.0,SPC,"0,339009",ОБЛ СВЕРДЛОВСКАЯ,38000.0


In [ ]:
df.shape

(114399, 15)

kNN

In [ ]:
knn = KNeighborsClassifier()

params = {
    'n_neighbors': np.arange(1, 20)
}

gs = GridSearchCV(knn, params, scoring='f1_macro')

In [ ]:
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

(0.5067632315985726, {'n_neighbors': 1})

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-48-7d9e95e2df28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


Random Forest

In [ ]:
rfc = RandomForestClassifier()

params = {
    'n_estimators': np.arange(20, 30, 2),
    'max_depth': np.arange(10, 20, 2)
}

gs = GridSearchCV(rfc, params, scoring='f1_macro')

In [ ]:
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

(0.5308038506578823, {'max_depth': 18, 'n_estimators': 26})

In [ ]:
rfc = RandomForestClassifier(max_depth=18, n_estimators=26)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-42-7d9e95e2df28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


sgdc classifier

In [ ]:
sgdc = SGDClassifier().fit(X_train, y_train)
y_pred = sgdc.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-63-7d9e95e2df28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier().fit(X_train, y_train)
y_pred = gb.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-50-7d9e95e2df28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


lgbm boosting

In [ ]:
lgbm = light.LGBMClassifier().fit(X_train, y_train)
y_pred = lgbm.predict(X_test)

[LightGBM] [Info] Number of positive: 20177, number of negative: 94222
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004177 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1115
[LightGBM] [Info] Number of data points in the train set: 114399, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.176374 -> initscore=-1.541110
[LightGBM] [Info] Start training from score -1.541110


In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-52-7d9e95e2df28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


xgb

In [ ]:
grid = {
    'n_estimators' : [100, 150, 200],
    'learning_rate': [0.1, 0.2, 0.5],
}
xg = xgb.XGBClassifier()
gs = GridSearchCV(xg, grid, n_jobs = -1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2, 0.5],
                         'n_estimators': [100, 150, 200]})

In [ ]:
gs.best_score_, gs.best_params_

(0.828241510063432, {'learning_rate': 0.1, 'n_estimators': 100})

In [ ]:
xg = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1).fit(X_train, y_train)
y_pred = xg.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-61-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


cat

In [ ]:
ct = cat.CatBoostClassifier().fit(X_train, y_train)
y_pred = ct.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-57-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


ada

In [ ]:
ada = AdaBoostClassifier().fit(X_train, y_train)
y_pred = ada.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-63-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


naive bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-70-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

In [ ]:
nb = BernoulliNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-19-8c4913529559>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


In [ ]:
nb = ComplementNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

ValueError: Negative values in data passed to ComplementNB (input X)

In [ ]:
nb = CategoricalNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

ValueError: Negative values in data passed to CategoricalNB (input X)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

ValueError: Negative values in data passed to MultinomialNB (input X)

svc

In [ ]:
svc = SVC(kernel='sigmoid')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-76-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,True
3,23504,False
4,69269,False


kmeans

In [ ]:
grid = {
    'n_clusters' : np.arange(4, 15)
}
km = KMeans()
gs = GridSearchCV(km, grid, n_jobs=-1)
gs.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


GridSearchCV(estimator=KMeans(), n_jobs=-1,
             param_grid={'n_clusters': array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])})

In [ ]:
gs.best_score_, gs.best_params_

(-9417378346221.36, {'n_clusters': 14})

In [ ]:
km = KMeans(n_clusters=14)
km.fit(X_train, y_train)
y_pred = km.predict(X_test)

In [ ]:
params = {
    'n_estimators': np.arange(5, 15),
    'max_samples': np.arange(1, 3, 1),

}

gs = GridSearchCV(BaggingClassifier(KMeans(n_clusters=14)), params, scoring='f1_macro')
gs.fit(X_train, y_train)

In [ ]:
gs.best_score_, gs.best_params_

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

decision tree

In [ ]:
tree = DecisionTreeClassifier()

params = {
    'max_depth': np.arange(1, 10),
}

gs = GridSearchCV(tree, params, scoring='f1_macro')

In [ ]:
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

(0.5269154125625286, {'max_depth': 9})

In [ ]:
tree = DecisionTreeClassifier(max_depth=9)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-74-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


mlp

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32, ), activation='tanh', solver='adam', alpha=0.01)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-70-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,False
1,10673,False
2,121409,False
3,23504,False
4,69269,False


voting

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
km = KMeans(n_clusters=14)
rfc = RandomForestClassifier(max_depth=18, n_estimators=26)
svc = SVC(kernel='poly')
lr = LinearRegression()


vt = VotingClassifier(
    	estimators =[('knn', knn), ('svc', svc), ('rfc', rfc)],
    	voting='hard')
vt.fit(X_train, y_train)

In [ ]:
y_pred = vt.predict(X_test)

In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_dim=X_train.shape[1], activation='exponential'))
model.add(Dense(2, activation='exponential'))
model.add(Dense(1, activation='exponential'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=512)
y_pred = model.predict(X_test)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


224/224 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.8236
Epoch 2/5
224/224 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.8236
Epoch 3/5
224/224 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.8236
Epoch 4/5
224/224 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.8236
Epoch 5/5
1761/1761 [==============================] - 2s 1ms/step


In [ ]:
df_sub = df_test[['id',]]
df_sub['opened'] = y_pred.astype(bool)
df_sub.to_csv('my_submission.csv', index=True)
df_sub.head()

<ipython-input-54-2499c41809df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['opened'] = y_pred.astype(bool)


,id,opened
0,39162,True
1,10673,True
2,121409,True
3,23504,True
4,69269,True
